In [1]:
import ollama
from ollama import chat

from huggingface_hub import snapshot_download

from docling.datamodel.pipeline_options import PdfPipelineOptions, RapidOcrOptions
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.chunking import HybridChunker

from pydantic.v1 import BaseModel

from transformers import AutoTokenizer
from sentence_transformers import CrossEncoder

from BM25 import load_bm25, create_bm25, bm25_search
from util import docling_util
from util.qdrant_util import qdrant_DBConnector, DataObject
from util.text_splitter import RecursiveTextSplitter, DataFrameFormatter
from util.ollama_util import *

import os
import json

/Users/yoyo/opt/anaconda3/envs/cathay_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# document local path or URL
doc = "./test_doc/test_textMore.pdf" # simple text with a image and table
doc1 = "./test_doc/吉美利101利率變動型美元終身壽險(定期給付型)DM.pdf" # complex text with simple aesthetic table
doc2 = "./test_doc/美利雙寶利率變動型美元終身保險 (定期給付型)DM.pdf" # complex text with complex aesthetic table
doc3 = "./test_doc/國泰金控出勤管理須知(修正後).pdf" # simple text with a simple table
doc4 = "./test_doc/國泰金控員工國內出差要點.pdf" # simple text with a simple table
doc5 = "./test_doc/國泰金控員工國外出差要點.pdf" # simple text with complex table
doc6 = "./test_doc/國泰金融控股股份有限公司資訊安全管理辦法.pdf" # simple text
doc7 = "./test_doc/國泰金融控股股份有限公司_後台維護系統管理作業要點_F.pdf" # flow map with blank table
doc8 = "./test_doc/國泰金融控股股份有限公司金融科技創新業務管理辦法_20240125.pdf" # flow map
doc9 = "./test_doc/國泰金控暨子公司「對話機器人 - 阿發」視覺形象管理辦法.pdf" # complex figure design doc
doc10 = "./test_doc/國泰醫院113年工作計畫.pdf" # cross page table and HAS index, scanned document OCR needed
doc11 = "./test_doc/國泰醫院113年經費預算.pdf" # large number table, scanned document OCR needed
doc12 = "./test_doc/國泰醫院112年工作報告.pdf" # table with check board inside, scanned document OCR needed
doc13 = "./test_doc/112年國泰醫療財團法人財務報告.pdf" # loose structured table, scanned document OCR needed
doc14 = "./test_doc/111年4-6月愛心捐款及捐贈物資徵信名冊.pdf" #compacted table
doc15 = "./test_doc/畢業學分審核表.pdf" # vertical table

# ollama embedding model
embed_model = "bge-m3:latest"

# reranker 
#rerank_model = CrossEncoder('BAAI/bge-reranker-large', max_length=512)
#rerank_model = CrossEncoder('BAAI/bge-reranker-v2-m3', max_length=8192)
rerank_model = CrossEncoder('BAAI/bge-reranker-v2-m3', max_length=1024)

# ollama llm
#llm = "deepseek-r1:7b"
#llm = "deepseek-r1:14b"
#llm = "deepseek-r1:32b"
#llm = "deepseek-r1:14b-max-context"
#llm = "deepseek-r1:14b-quarter-context"
llm = "deepseek-r1:14b-eighth-context"

"""def get_embeddings(texts, model=embed_model):
    embed_response = ollama.embeddings(model=model, prompt=texts)
    embedded_vector = embed_response["embedding"]
    
    return embedded_vector"""

# PyPdfium without EasyOCR
# --------------------
"""pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False
#pipeline_options.table_structure_options.do_cell_matching = True

#pipeline_options.table_structure_options.mode = 'accurate'

pipeline_options.images_scale = 2
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True

pypdfium_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
        pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
        )
    }
)"""

# PyPdfium with RapidOCR
# ----------------------
# Download RappidOCR models from HuggingFace
print("Downloading RapidOCR models")
download_path = snapshot_download(repo_id="SWHL/RapidOCR")

det_model_path = os.path.join(
    download_path, "PP-OCRv4", "ch_PP-OCRv4_det_infer.onnx"
)
rec_model_path = os.path.join(
    download_path, "PP-OCRv4", "ch_PP-OCRv4_rec_infer.onnx"
)
cls_model_path = os.path.join(
    download_path, "PP-OCRv3", "ch_ppocr_mobile_v2.0_cls_train.onnx"
)
ocr_options = RapidOcrOptions(
    det_model_path=det_model_path,
    rec_model_path=rec_model_path,
    cls_model_path=cls_model_path,
    #force_full_page_ocr=True
)

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
#pipeline_options.table_structure_options.do_cell_matching = False
pipeline_options.table_structure_options.do_cell_matching = True
pipeline_options.table_structure_options.mode = 'accurate'

pipeline_options.images_scale = 2
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True

pipeline_options.ocr_options = ocr_options

pypdfium_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
        pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
        )
   }
)

# print json
def show_json(data):
    if isinstance(data, str):
        obj = json.loads(data)
        print(json.dumps(obj, indent=4, ensure_ascii=False))
    elif isinstance(data, dict) or isinstance(data, list):
        print(json.dumps(data, indent=4, ensure_ascii=False))
    elif issubclass(type(data), BaseModel):
        print(json.dumps(data.dict(), indent=4, ensure_ascii=False))

Fetching 21 files: 100%|██████████| 21/21 [00:00<00:00, 70975.33it/s]


# Docling reader, docling format parser output

In [43]:
doc_source = [doc, doc1, doc2, doc3, doc4, doc5, doc6]                         # original test docs
#doc_source = [doc4, doc5, doc7, doc8, doc9, doc10, doc11, doc12, doc13, doc14] # with added docs
#doc_source = [doc10, doc11, doc12, doc13]                                      # scanned docs and loose table
#doc_source = [doc15]                                                            # vertical table
#doc_source = [doc, doc1, doc2, doc3, doc4, doc5, doc6, doc7, doc8, doc9, doc10, doc11, doc12, doc13, doc14]
#doc_source = [doc5]

conv_results = pypdfium_converter.convert_all(
    doc_source,
    raises_on_error=True,  # to let conversion run through all and examine results at the end
)

conv_results_list = list(conv_results)

# Do hybrid chunking to merge similar chunk
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

hybrid_chunker = HybridChunker(
    tokenizer=tokenizer,
    max_tokens=8000,
    merge_peers=True  # optional, defaults to True
)

# text chunks
all_chunks = []
for conv_res in conv_results_list:
    docling_docs = conv_res.document
    chunk_iter = hybrid_chunker.chunk(dl_doc=docling_docs)
    chunks = list(chunk_iter)
    all_chunks += chunks


In [44]:
for i, chunk in enumerate(all_chunks[:]):
    print(f"=== {i} ===")
    txt_tokens = len(tokenizer.tokenize(chunk.text))
    print(f"chunk.text ({txt_tokens} tokens):\n{repr(chunk.text)}")

    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")

    print()

=== 0 ===
chunk.text (285 tokens):
'（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 . 12 . 31 依 113.09.23 金管保壽字第 1130427324 號4 號函修正'
chunker.serialize(chunk) (311 tokens):
'國泰人壽祿壽祿美富利富利率變動型美元終身壽險（定期給付型）\n（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 

# Rechunk large chunks with RecursiveTextSplitter

In [45]:
big_chunk = []

for i, chunk in enumerate(all_chunks[:]):
    
    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    if ser_tokens > 1024:
        print(f"=== {i} ===")
        #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
        print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
        big_chunk.append(chunk)
        print()
    else:
        pass

=== 52 ===
chunker.serialize(chunk) (1188 tokens):
揭露事項
本保險各保單年度末之「解約金及累積生存金總和」與「應繳保險費累積值」之差異 情形揭露如下(相關資訊可至國泰人壽官方網站資訊公開頁面查詢)：
1, 男性.5歲 = 38%. 1, 男性.35歲 = 39%. 1, 男性.65歲 = 36%. 1, 女性.5歲 = 38%. 1, 女性.35歲 = 39%. 1, 女性.65歲 = 36%. 5, 男性.5歲 = 65%. 5, 男性.35歲 = 65%. 5, 男性.65歲 = 60%. 5, 女性.5歲 = 64%. 5, 女性.35歲 = 65%. 5, 女性.65歲 = 61%. 10, 男性.5歲 = 92%. 10, 男性.35歲 = 92%. 10, 男性.65歲 = 83%. 10, 女性.5歲 = 92%. 10, 女性.35歲 = 92%. 10, 女性.65歲 = 85%. 15, 男性.5歲 = 96%. 15, 男性.35歲 = 94%. 15, 男性.65歲 = 82%. 15, 女性.5歲 = 96%. 15, 女性.35歲 = 96%. 15, 女性.65歲 = 85%. 20, 男性.5歲 = 100%. 20, 男性.35歲 = 97%. 20, 男性.65歲 = 80%. 20, 女性.5歲 = 101%. 20, 女性.35歲 = 99%. 20, 女性.65歲 = 85%
註1：上表數值係以基本保險金額為基礎進行試算，且累積生存金與應繳保險費累積值已加 計利息，計息基礎為前-日曆年度之十二個月臺灣銀行、第-銀行與合作金庫三家行 庫每月初（每月第-個營業日）牌告之二年期定期儲蓄存款最高年利率之平均值 （1.72%）。
註2：上表數值若比例小於100%，表示投保後提早解約或不繼續繳費，將可能產生不利消 費者之情形。
第2頁，共2頁，2025年1月版
1.消費者投保前應審慎瞭解本商品之承保範圍、除外責任、不保事項及商品風險，相關內容均詳列於保 單條款及相關銷售文件，如有疑義請洽詢銷售人員以詳細說明。
2. 本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目。
3.要保人可透過國泰人壽客服專線(市話免費撥打：0800-036-599、付費撥打： 02-2162-6201)或

In [46]:
splitter = RecursiveTextSplitter(tokenizer=tokenizer, max_tokens=1024, overlap=150, min_length_ratio=1)

rechunked_large_chunks = []

for i, chunk in enumerate(big_chunk):
    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    if splitter.tokenize_len(ser_txt) > splitter.max_tokens:
        sub_chunks = splitter.split_text(ser_txt)
        sub_chunks_meta = {"filename": chunk.meta.origin.filename}
        sub_chunks_pair = [(text, sub_chunks_meta) for text in sub_chunks]
        rechunked_large_chunks.extend(sub_chunks_pair)
    else:
        rechunked_large_chunks.append(ser_txt)


"""ser_txt = hybrid_chunker.serialize(chunk=rechunked_large_chunks[1])
if splitter.tokenize_len(ser_txt) > splitter.max_tokens:
    sub_chunks = splitter.split_text(ser_txt)
    rechunked_large_chunks.extend(sub_chunks)
else:
    rechunked_large_chunks.append(ser_txt)"""

'ser_txt = hybrid_chunker.serialize(chunk=rechunked_large_chunks[1])\nif splitter.tokenize_len(ser_txt) > splitter.max_tokens:\n    sub_chunks = splitter.split_text(ser_txt)\n    rechunked_large_chunks.extend(sub_chunks)\nelse:\n    rechunked_large_chunks.append(ser_txt)'

In [47]:
for i, chunk in enumerate(rechunked_large_chunks[:]):
    
    ser_txt = chunk[0]
    ser_tokens = len(tokenizer.tokenize(ser_txt))

    print(f"=== {i} ===")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
    print(chunk[1])

    print()

=== 0 ===
chunker.serialize(chunk) (970 tokens):
揭露事項
本保險各保單年度末之「解約金及累積生存金總和」與「應繳保險費累積值」之差異 情形揭露如下(相關資訊可至國泰人壽官方網站資訊公開頁面查詢)：
1, 男性.5歲 = 38%. 1, 男性.35歲 = 39%. 1, 男性.65歲 = 36%. 1, 女性.5歲 = 38%. 1, 女性.35歲 = 39%. 1, 女性.65歲 = 36%. 5, 男性.5歲 = 65%. 5, 男性.35歲 = 65%. 5, 男性.65歲 = 60%. 5, 女性.5歲 = 64%. 5, 女性.35歲 = 65%. 5, 女性.65歲 = 61%. 10, 男性.5歲 = 92%. 10, 男性.35歲 = 92%. 10, 男性.65歲 = 83%. 10, 女性.5歲 = 92%. 10, 女性.35歲 = 92%. 10, 女性.65歲 = 85%. 15, 男性.5歲 = 96%. 15, 男性.35歲 = 94%. 15, 男性.65歲 = 82%. 15, 女性.5歲 = 96%. 15, 女性.35歲 = 96%. 15, 女性.65歲 = 85%. 20, 男性.5歲 = 100%. 20, 男性.35歲 = 97%. 20, 男性.65歲 = 80%. 20, 女性.5歲 = 101%. 20, 女性.35歲 = 99%. 20, 女性.65歲 = 85%
註1：上表數值係以基本保險金額為基礎進行試算，且累積生存金與應繳保險費累積值已加 計利息，計息基礎為前-日曆年度之十二個月臺灣銀行、第-銀行與合作金庫三家行 庫每月初（每月第-個營業日）牌告之二年期定期儲蓄存款最高年利率之平均值 （1.72%）。
註2：上表數值若比例小於100%，表示投保後提早解約或不繼續繳費，將可能產生不利消 費者之情形。
第2頁，共2頁，2025年1月版
1.消費者投保前應審慎瞭解本商品之承保範圍、除外責任、不保事項及商品風險，相關內容均詳列於保 單條款及相關銷售文件，如有疑義請洽詢銷售人員以詳細說明。
2. 本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目。
3.要保人可透過國泰人壽客服專線(市話免費撥打：0800-036-599、付費撥打： 02-2162-6201)或網站

# Or table extraction result

In [48]:
# table chunks
all_tables = docling_util.extract_tables(conv_results_list)

"""for table_ix, table in enumerate(all_tables):
    print(f"## Table {table_ix}")
    display(table[0])"""

'for table_ix, table in enumerate(all_tables):\n    print(f"## Table {table_ix}")\n    display(table[0])'

In [49]:
table_formatter = DataFrameFormatter(tokenizer=tokenizer, show_index=False, max_tokens=1024)

table_chunks = []
for table in all_tables:
    chunks = table_formatter.chunk_rows(table[0])
    chunks_pair = [(text, table[1]) for text in chunks]
    table_chunks.extend(chunks_pair)

In [50]:
for i, chunk in enumerate(table_chunks[:]):
    
    ser_txt = chunk[0]
    ser_tokens = len(tokenizer.tokenize(ser_txt))

    print(f"=== {i} ===")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
    print(chunk[1])
    print()

=== 0 ===
chunker.serialize(chunk) (92 tokens):
0 = 保險年齡, 1 = 30歲以下, 2 = 31歲至40歲, 3 = 41歲至50歲, 4 = 51歲至60歲, 5 = 61歲至70歲, 6 = 71歲至90歲, 7 = 91歲以上
0 = 比率, 1 = 190%, 2 = 160%, 3 = 140%, 4 = 120%, 5 = 110%, 6 = 102%, 7 = 100%
{'filename': 'test_textMore.pdf'}

=== 1 ===
chunker.serialize(chunk) (271 tokens):
項別 = -, 失 能  程  度 = 雙目均失明者（註1）。
項別 = 二, 失 能  程  度 = 兩上肢腕關節缺失者或兩下肢足踝關節缺失者。
項別 = 三, 失 能  程  度 = -上肢腕關節及-下肢足踝關節缺失者。
項別 = 四, 失 能  程  度 = -目失明及-上肢腕關節缺失者或-目失明及-下肢足踝關節缺失者。
項別 = 五, 失 能  程  度 = 永久喪失咀嚼（註（註2）或言語（註3）之機能者。
項別 = 六, 失 能  程  度 = 四肢機能永久完全喪失者（註（註4） 。
項別 = 七, 失 能  程  度 = 中樞神經系統機能遺存極度障害或胸、腹部臟器機能遺存極度障害，終身不能從事任何工作，經常 需醫療護理或專人周密照護者（註（註5） 。
{'filename': 'test_textMore.pdf'}

=== 2 ===
chunker.serialize(chunk) (136 tokens):
保險單年度 = 第 1 年, 可借成數 = 60%, 可借金額上限 = 可借金額上限＝借款當日保單價值準備金 × 可借成數
保險單年度 = 第 2~6 年, 可借成數 = 70%, 可借金額上限 = 可借金額上限＝借款當日保單價值準備金 × 可借成數
保險單年度 = 第 7 年及以後, 可借成數 = 85%, 可借金額上限 = 可借金額上限＝借款當日保單價值準備金 × 可借成數
{'filename': 'test_textMore.pdf'}

=== 3 ===
chunker.serialize(chunk) (73 tokens):
單

# Merge all chunks

In [51]:
all_chunks.extend(rechunked_large_chunks)
all_chunks.extend(table_chunks)
all_chunks

[DocChunk(text='（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 . 12 . 31 依 113.09.23 金管保壽字第 1130427324 號4 號函修正', meta=DocMeta(schema_name='docling_core.transforms.chunker.DocMeta', version='1.0.0', doc_items=[DocItem(self_ref='#/texts/1', parent=RefItem(cref='#/groups/0'), children=[], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.TEXT: 'text'>, prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=61.619998931884766, t=745.7193603515625, r=391.8338623046875, b=736.2374267578125, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 34))]), DocItem(self_ref='#/texts/2', parent=RefItem(cref='

In [54]:
node_text, node_metadatas = [], []
for chunk in all_chunks:
    if type(chunk) == tuple:
        node_text.append(chunk[0])
        node_metadatas.append(json.dumps(chunk[1], indent=4, ensure_ascii=False))
    else:
        node_text.append(hybrid_chunker.serialize(chunk=chunk))
        node_metadatas.append(chunk.meta.export_json_dict())

In [15]:
embedded_text = []
for i in node_text:
    embedded_text.append(get_embeddings(i))

file_id = 123
node_text, node_metadatas = [], []

for chunk in all_chunks:
    if type(chunk) == tuple:
        node_text.append(chunk[0])
        meta_dict = chunk[1].copy()
        meta_dict["file_id"] = file_id
        #node_metadatas.append(json.dumps(meta_dict, indent=4, ensure_ascii=False))
        node_metadatas.append(meta_dict)
    else:
        node_text.append(hybrid_chunker.serialize(chunk=chunk))
        meta_dict = chunk.meta.export_json_dict()
        meta_dict["file_id"] = file_id
        #node_metadatas.append(json.dumps(meta_dict, indent=4, ensure_ascii=False))
        node_metadatas.append(meta_dict)

node_metadatas

embedded_tables = []
for i in all_tables_chunks:
    embedded_tables.append(get_embeddings(i))

# Test on BM25 retrieval

In [16]:
#query = "去新加坡出差可以申請多少費用？"
#query = "科主管國內出差可以申請多少餐雜宿費？"
query = "系統開發之安全管理，應包含哪些項目？"

output_dir = 'test_BM25index'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# as json
bm25_mixed_json = create_bm25(node_text, 'mixed')
bm25_mixed_json.save(os.path.join(output_dir, 'bm25_mixed.json'))

loaded_bm25_mixed_json = load_bm25(os.path.join(output_dir, 'bm25_mixed.json'), node_text)

print("load from json and mixure search:", query)
print()
results_json = loaded_bm25_mixed_json.search(query, top_k=5)
for doc_id, score in results_json:
    print(f"Doc ID: {doc_id}, Score: {score:.4f}, Text: {node_text[doc_id]}\n")

Building prefix dict from /Users/yoyo/Documents/國泰/BM25/dict.txt.big ...
Loading model from cache /var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/jieba.u938b08ca6a75588702a8549caaec8c7e.cache
Loading model cost 0.753 seconds.
Prefix dict has been built successfully.


load from json and mixure search: 系統開發之安全管理，應包含哪些項目？

Doc ID: 130, Score: 25.3129, Text: 使用外部即時通訊軟體,亦 不得使用網頁或其他替代方式進行電腦即時通訊。 第十條 本公司資司資訊系統存取控制之相關作業,資,資訊處、數數發中心或心或受委託 者應者應依下列規定辦理: -、 資訊系統之管理帳號 與使用帳號應 有權責區分 且制定規 範原則,規範中範中需包含帳含帳號申請、異、異動程序,並留存申請紀錄。 二、 非本公司員工之帳號及權限應於合約終止時予以調整、停用或 刪除。 三、 伺服器特殊權限帳號,如具有作業系統管理權限、特殊資料存取 權限、其它系統資源控制權限及存取稽核軌跡之帳號,其使用應 僅限於被授權核准之事項,留存適當之稽核軌跡並定期進行檢 視。 四、 每-經申請核准之使用者應有專屬且獨立的帳號及密碼,且每 年應定期清查覆核。 五、 密碼之碼之安全管理,應包含下列項目: (-) 密碼長度、複雜性、密碼歷程之程之要求。 (二) 密碼定期變更及錯誤鎖定之要求。 本公司網路暨通訊管理之相關作業,應依下列規定辦理: -、 帳號使用單位、資訊處、數數發中心或受委受委託者應者應定期審視分工 區隔作業以及各帳號授權的妥適性,離職、調職、停職、留職停 薪人員之權限應於生效日(含(含)前)前取消或停用。 二、 帳號使用者對所持有之帳號、密碼與權限應善盡保管責任並於 授權範圍內妥善使用 。 第十-條 本公司系統開發及維護之護之安全管理,資訊處、數數發中心或心或 受委託者應者應依下列規定辦理: -、 系統開發應於初始階段考量安控機制之布置,委託委託開託開發部分應 第 6 頁6 頁/共 8 頁8 頁 強化控管資訊安全並於合約載明相關義務, 系統開發應注意時 程之掌控,避免延誤脫序。 二、 系統開統開發之安全管理,應包含下列項目: (-) 需求提出階段之安全管理。 (二) 系統分析階段之安全管理。 (三) 系統設計階段之安全管理。 (四) 系統測試階段之安全管理。 (五) 系統驗收及上線階段之安全管理。 三、 系統變更之安全管理,應包含下列項目: (-) 原始碼之安全管理。 (二) 程式佈署之安全管理。 (三) 緊) 緊急變更之安全管理。 四、 測試環境與正式環境應分別獨立於不同電腦作業環境。 第十二條 

# Qdrant


from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import PointStruct

class qdrant_DBConnector:
    def __init__(self, collection_name):#, embedding_fn):
        self.qdrant_client = QdrantClient("http://localhost:6333")

        self.collection_name = collection_name

        
        """# create collection
        self.collection = self.qdrant_client.recreate_collection(
            collection_name = collection_name,
            vectors_config = models.VectorParams(
                distance = models.Distance.COSINE,
                size=len(get_embeddings("你好"))),
            optimizers_config = models.OptimizersConfigDiff(memmap_threshold=20000),
            hnsw_config = models.HnswConfigDiff(on_disk=True, m=16, ef_construct=100)
        )"""
        
        
        '''
        # embed models
        self.embedding_fn = embedding_fn
        qdrant_client.set_model(self.embedding_fn)
        '''

    ''' WARNING: NO CHECK ON EQUAL LENGTH YET'''
    def upsert_vector(self, vectors, data):
        # insert 'points' to qdrant by vector, 
        # payload with original text and metadata
        for i, vector in enumerate(vectors):
            """ WARNING: SHOULD CHECK DIMENSION==EMBEDDING_DIMENSION INSTEAD"""
            if len(vector) == 0:
                continue
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=[PointStruct(id=i,
                                    vector=vectors[i],
                                    payload={"text": data.text[i],
                                             "metadata": data.metadata[i]})]
                                    
            )

        print("upsert finish")

    def retrieved_all(self):
        count_points = self.qdrant_client.count(
            collection_name=self.collection_name,
            exact=True,
        )
        result = self.qdrant_client.retrieve(
            collection_name=self.collection_name,
            ids=list(range(0, count_points.count)),
        )
        return result

    def vector_search(self, vector, top_k):
        # vector search qdrant DB
        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            limit=top_k,
            append_payload=True,
        )
        return result
    
    def vector_search_json(self, vector, top_k):
        # vector search qdrant DB with json format output
        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            limit=top_k,
            append_payload=True,
        )
        vector_result_json = {
            f"chunk_{item.id}": {
                "text": item.payload['text'],
                "rank": index,
                "score": item.score
            }
            for index, item in enumerate(result)
        }

        return vector_result_json

In [17]:
"""class DataObject:
    def __init__(self, text, metadata):
        self.text = text
        self.metadata = metadata if metadata else [{} for _ in range(len(text))]"""

data = DataObject(node_text, node_metadatas)
data.text[0]

'國泰人壽祿壽祿美富利富利率變動型美元終身壽險（定期給付型）\n（給付項目：祝壽保險金、身故保險金或喪葬費用保險金、完全失能保險金）\n（本保險提供身故保險金分期定期給付）\n（本保險為不分紅保險單，不參加紅利分配，並無紅利給付項目）\n（本契約與以新臺幣收付之人身保險契約間，不得辦理契約轉換）\n（本商品部分年齡可能發生累積所繳保險費超出身故保險金給付之情形）\n（申訴電話：市話免費撥打0800-036-599、付費撥打02-2162-6201；傳真：0800-211-568；電子信箱（E-mail）：\nservice@cathaylife.com.tw）\n110.08.12國2國壽字第1100080120號0號函備查\n111 . 12 . 01國1國壽字第1110120022號2號函備查\n112.02.23依3依111.11.29金管保壽字第1110462568號8號函修正\n113.07.01 依 113.06.27 金管保壽字第 11304921171 號1 號令修正\n113 . 12 . 31 依 113.09.23 金管保壽字第 1130427324 號4 號函修正'

In [18]:
# create a qdrant vectorDB object
#vector_db = qdrant_DBConnector("qdrant_result")
vector_db = qdrant_DBConnector("qdrant_new", recreate=True)

In [19]:
# add data to db
vector_db.upsert_vector(embedded_text, data)

upsert finish


table_datas = DataObject(all_tables_chunks, [])
# create a qdrant vectorDB object
vector_db_table = qdrant_DBConnector("qdrant_table")
# add data to db
vector_db_table.upsert_vector(embedded_tables, table_datas)

In [ ]:
#query = "國內出差費用標準"
#query = "國外出差，附表五 "
query = "系統開發之安全管理，應包含哪些項目？"

embedded_query = get_embeddings(query)
results = vector_db.vector_search(embedded_query, 5)
#results = vector_db_table.vector_search(embedded_query, 5)
print(f"尋找 「{query}」:")
#print(results[0])

for i, point in enumerate(results):
    print(f"\n🔹 Score: {point.score:.2f} - point ID: {point.id}:")
    print(point.payload['text'])
    print()


尋找 「系統開發之安全管理，應包含哪些項目？」:

🔹 Score: 0.41 - point ID: 8:
第八條(修(修訂及施行)
附件：國內出差雜、宿、交通費標準
人員/項/項目, 1 = 全日雜費. 人員/項/項目, 2 = 宿費. 人員/項/項目, 3 = 交通費. 總經理以上, 1 = 檢據實支. 總經理以上, 2 = 檢據實支. 總經理以上, 3 = 檢據實支. 高階主管, 1 = 檢據實支. 高階主管, 2 = 檢據實支. 高階主管, 3 = 檢據實支. 部室主管及相當職級, 1 = . 部室主管及相當職級, 2 = 4 , 000(5,500)元. 部室主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 科主管及相當職級, 1 = . 科主管及相當職級, 2 = 4 , 000(5,500)元. 科主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 其他人員, 1 = . 其他人員, 2 = 4 , 000(5,500)元. 其他人員, 3 = 車 費 車 費 用 單 程 上 限上 限


🔹 Score: 0.40 - point ID: 0:
國泰金融控股股份有限公司員工國內出差要點
940405 訂5 訂定
951031 修訂
960509 修9 修訂
980728 修8 修訂
1020101 修訂
1030206 修6 修訂
1030523 修3 修訂
1040904 修4 修訂
1070326 修6 修訂
1090101 修訂
1120801修1修訂
1130701修1修訂
權責單位：人力資源部


🔹 Score: 0.39 - point ID: 9:
第八條(修(修訂及施行)
備註：
-、本公司員工於國內出差時應檢具實支，金額上限如前表；無法檢具實支者，則以 金額上限支給。
二、本要點所稱「高階主管」係指部室主管(不含)以上，總經理(不含)以下之主管。
三、員工出差如搭乘飛機或高鐵，可縮短時間、減少雜費與宿費而節省出差旅費時， 可逕行搭乘(高(高階主管以上人員檢據實支，部室主管級以下人員飛機限搭經濟艙、 高鐵限搭標準車廂)。
四、括弧內金額為員工出差及住宿地為雙為雙北地區地區時之宿費核給標準。
五、部室主管級以

# Try chatting!

In [ ]:
"""stream = chat(
    model='deepseek-r1:7b',
    messages=[{'role': 'user', 'content': '妳好'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)"""

"stream = chat(\n    model='deepseek-r1:7b',\n    messages=[{'role': 'user', 'content': '妳好'}],\n    stream=True,\n)\n\nfor chunk in stream:\n  print(chunk['message']['content'], end='', flush=True)"

def bm25_retrieval(query, vector_db_name=vector_db, top_k=3):
    result = vector_db_name.retrieved_all()
    retrieved_text = []
    for i, points in enumerate(result):
        retrieved_text.append(result[i].payload['text'])

    bm25_result = bm25_search(corpus=retrieved_text, query=query, top_k=top_k)
    bm25_result_json = {
        f"chunk_{item[0]}": {
            "text": item[2],
            "rank": index,
            "score": item[1]
        }
        for index, item in enumerate(bm25_result)
    }

    return bm25_result_json

def rrf(ranks, k=1):
    ret = {}
    # recursive through all retrieved method
    for rank in ranks:
        for id, val in rank.items():
            if id not in ret:
                ret[id] = {"score": 0, "text": val["text"]}
            # calculate rrf score
            ret[id]["score"] += 1.0/(k+val["rank"])
    # sort and return according to rrf score
    return dict(sorted(ret.items(), key=lambda item: item[1]["score"], reverse=True))

def get_completion(prompt, model=llm):
    messages = [{"role": "user", "content": prompt}]
    response = chat(
        model=model,
        messages=messages
        #stream=True,
        #format="json",
        #options={"temperature":0}
    )
    return response.message.content


In [ ]:
query = "去中國大陸出差可以申請多少費用？"

print("search for:", query)
bm25_retrieve_result = bm25_retrieval(vector_db, query, top_k=3)
show_json(bm25_retrieve_result)

search for: 去中國大陸出差可以申請多少費用？
{
    "chunk_6": {
        "text": "第七條費(費條(費用核銷)\n員工公差完畢應即返回任所銷差，並於七日內於國內出差費用核銷系統提出費用申請，並檢 附相關收據，報請費用核銷。",
        "rank": 0,
        "score": 4.62196595598322
    },
    "chunk_5": {
        "text": "第六條(其他費用)\n公差人員交際應酬費用除呈奉准由公司開支之部份外，-概自理。",
        "rank": 1,
        "score": 2.8947026914120833
    },
    "chunk_3": {
        "text": "第三條(出差申請程序)\n員工因公需要出差時，應事先填具出差申請單，詳填出差預定日程，經該單位主管核簽並依 分層負責表規定呈主管核准後始得出差，並依員工出勤管理要點相關規定辦理。",
        "rank": 2,
        "score": 2.0137874080220834
    }
}


In [ ]:
embedded_query = get_embeddings(query)
vector_result_json = vector_db.vector_search_json(embedded_query, 3)
print("search for:", query)

show_json(vector_result_json)

search for: 去中國大陸出差可以申請多少費用？
{
    "chunk_8": {
        "text": "第八條(修(修訂及施行)\n附件：國內出差雜、宿、交通費標準\n人員/項/項目, 1 = 全日雜費. 人員/項/項目, 2 = 宿費. 人員/項/項目, 3 = 交通費. 總經理以上, 1 = 檢據實支. 總經理以上, 2 = 檢據實支. 總經理以上, 3 = 檢據實支. 高階主管, 1 = 檢據實支. 高階主管, 2 = 檢據實支. 高階主管, 3 = 檢據實支. 部室主管及相當職級, 1 = . 部室主管及相當職級, 2 = 4 , 000(5,500)元. 部室主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 科主管及相當職級, 1 = . 科主管及相當職級, 2 = 4 , 000(5,500)元. 科主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 其他人員, 1 = . 其他人員, 2 = 4 , 000(5,500)元. 其他人員, 3 = 車 費 車 費 用 單 程 上 限上 限",
        "rank": 0,
        "score": 0.65292966
    },
    "chunk_9": {
        "text": "第八條(修(修訂及施行)\n備註：\n-、本公司員工於國內出差時應檢具實支，金額上限如前表；無法檢具實支者，則以 金額上限支給。\n二、本要點所稱「高階主管」係指部室主管(不含)以上，總經理(不含)以下之主管。\n三、員工出差如搭乘飛機或高鐵，可縮短時間、減少雜費與宿費而節省出差旅費時， 可逕行搭乘(高(高階主管以上人員檢據實支，部室主管級以下人員飛機限搭經濟艙、 高鐵限搭標準車廂)。\n四、括弧內金額為員工出差及住宿地為雙為雙北地區地區時之宿費核給標準。\n五、部室主管級以下人員隨同高階主管以上人員出差時，宿費得比照實支，但以投宿 同-飯店之較低標準房型為限。\n六、超過中午 12 點2 點得申請半日雜費、超過 18 點8 點則以全日計。",
        "rank": 1,
        "score": 0.63387895
   

In [ ]:
hybrid_result = rrf([vector_result_json, bm25_retrieve_result])

print(json.dumps(hybrid_result, indent=4, ensure_ascii=False))

{
    "chunk_8": {
        "score": 1.0,
        "text": "第八條(修(修訂及施行)\n附件：國內出差雜、宿、交通費標準\n人員/項/項目, 1 = 全日雜費. 人員/項/項目, 2 = 宿費. 人員/項/項目, 3 = 交通費. 總經理以上, 1 = 檢據實支. 總經理以上, 2 = 檢據實支. 總經理以上, 3 = 檢據實支. 高階主管, 1 = 檢據實支. 高階主管, 2 = 檢據實支. 高階主管, 3 = 檢據實支. 部室主管及相當職級, 1 = . 部室主管及相當職級, 2 = 4 , 000(5,500)元. 部室主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 科主管及相當職級, 1 = . 科主管及相當職級, 2 = 4 , 000(5,500)元. 科主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 其他人員, 1 = . 其他人員, 2 = 4 , 000(5,500)元. 其他人員, 3 = 車 費 車 費 用 單 程 上 限上 限"
    },
    "chunk_6": {
        "score": 1.0,
        "text": "第七條費(費條(費用核銷)\n員工公差完畢應即返回任所銷差，並於七日內於國內出差費用核銷系統提出費用申請，並檢 附相關收據，報請費用核銷。"
    },
    "chunk_9": {
        "score": 0.5,
        "text": "第八條(修(修訂及施行)\n備註：\n-、本公司員工於國內出差時應檢具實支，金額上限如前表；無法檢具實支者，則以 金額上限支給。\n二、本要點所稱「高階主管」係指部室主管(不含)以上，總經理(不含)以下之主管。\n三、員工出差如搭乘飛機或高鐵，可縮短時間、減少雜費與宿費而節省出差旅費時， 可逕行搭乘(高(高階主管以上人員檢據實支，部室主管級以下人員飛機限搭經濟艙、 高鐵限搭標準車廂)。\n四、括弧內金額為員工出差及住宿地為雙為雙北地區地區時之宿費核給標準。\n五、部室主管級以下人員隨同高階主管以上人員出差時，宿費得比照實支，但以投宿 同-飯店之較

def hybrid_retriever(query, top_k=3):
    embedded_query = get_embeddings(query)
    result = rrf([vector_db.vector_search_json(embedded_query, top_k), bm25_retrieval(query, top_k=top_k)])
    return result

def reranker(query, retrieved_result, threshold=0):
    text_chunks = []
    for chunk_id, val in retrieved_result.items():
        text_chunks.append(val['text'])
    scores = rerank_model.predict([(query, doc) for doc in text_chunks])
    sorted_list = sorted(zip(scores, text_chunks), key=lambda x: x[0], reverse=True)
    reranked_result = [chunk for chunk in sorted_list if chunk[0] > threshold]
    if len(reranked_result) < 3:
        reranked_result = sorted_list[:3]
    elif len(reranked_result) > 5:
        reranked_result = reranked_result[:5]

    return reranked_result
    #return [chunk for chunk in sorted_list if chunk[0] > threshold]

def format_rag_output(reranked_list):
    formatted_docs = "\n\n".join([f"Document {i+1}:\n{text[1]}" for i, text in enumerate(reranked_list)])
    return formatted_docs

def hybrid_retriever_with_BM25table(query, top_k=3):
    embedded_query = get_embeddings(query)
    result = rrf([vector_db.vector_search_json(embedded_query, top_k), bm25_retrieval(query, top_k=top_k), bm25_retrieval(query, vector_db_table, top_k)])
    return result

In [ ]:
#query = "去南非共和國出差可以申請多少費用？"
query = "重要會計項目中，現金及約當現金的銀行存款是多少？"

#print(format_rag_output(reranker(query, hybrid_retriever(query, 20), 0.45)))
print(format_rag_output(reranker(query, hybrid_retriever(vector_db, query, top_k=35), threshold=0.45)))

print()
#for chunk in reranker(query, hybrid_retriever(query, 20)):
for chunk in reranker(query, hybrid_retriever(vector_db, query, top_k=35)):
    print(chunk)
    print()

Document 1:
0 = 人員/項/項目, 1 = 全日雜費, 2 = 宿費, 3 = 交通費
0 = 總經理以上, 1 = 檢據實支, 2 = 檢據實支, 3 = 檢據實支
0 = 高階主管, 1 = 檢據實支, 2 = 檢據實支, 3 = 檢據實支
0 = 部室主管及相當職級, 2 = 4 , 000(5,500)元, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自
0 = 科主管及相當職級, 2 = 4 , 000(5,500)元, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自
0 = 其他人員, 2 = 4 , 000(5,500)元, 3 = 車 費 車 費 用 單 程 上 限上 限

Document 2:
第八條(修(修訂及施行)
附件：國內出差雜、宿、交通費標準
人員/項/項目, 1 = 全日雜費. 人員/項/項目, 2 = 宿費. 人員/項/項目, 3 = 交通費. 總經理以上, 1 = 檢據實支. 總經理以上, 2 = 檢據實支. 總經理以上, 3 = 檢據實支. 高階主管, 1 = 檢據實支. 高階主管, 2 = 檢據實支. 高階主管, 3 = 檢據實支. 部室主管及相當職級, 1 = . 部室主管及相當職級, 2 = 4 , 000(5,500)元. 部室主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 科主管及相當職級, 1 = . 科主管及相當職級, 2 = 4 , 000(5,500)元. 科主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 其他人員, 1 = . 其他人員, 2 = 4 , 000(5,500)元. 其他人員, 3 = 車 費 車 費 用 單 程 上 限上 限

Document 3:
國泰金融控股股份有限公司員工國內出差要點
940405 訂5 訂定
951031 修訂
960509 修9 修訂
980728 修8 修訂
1020101 修訂
1030206 修6 修訂
1030523 修3 修訂
1040904 修4 修訂
1070326 修6 修訂
1090101 修訂
1120801修1

In [ ]:
# Job instruction
instruction = """
你是台灣國泰金控的聊天機器人秘書，專門為用戶解答公司內部的差旅費用、報銷標準等相關問題。
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
3. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。
"""

# User input
#input_text = "去中國大陸出差可以申請多少費用？"
#input_text = "去南非共和國出差可以申請多少費用？"
input_text = "去智利出差可以申請多少費用？"
#input_text = "科主管國內出差可以申請多少餐雜宿費？"
#input_text = "商品貨幣是什麼？"
#input_text = "系統開發之安全管理，應包含哪些項目？"
#input_text = "國泰優惠APP 後台維護系統，有什麼功能？"
#input_text = "國泰優惠APP 後台維護系統，功能權限申請流程為何？" # Fail, didn't have the ability to recognize flow chart
#input_text = "國泰優惠 APP後台維護系統使用者權限表，有什麼欄位？"
#input_text = "金融科技創新業務之立案及概念測試流程為何？"  # Fail? didn't recognize flow chart but there's explaination on previous page
#input_text = "設計阿發時有什麼背景色彩限制？" # Fail? on picture detail explaination, but did catch the words on pics
#input_text = "根據111年度4-6月份捐款及捐贈物資徵信名冊，誰捐了鵝肉湯？" # doc14 fail on parsing compact table index, but seems like it does not interfere with llm answer
#input_text = "根據111年度4-6月份捐款及捐贈物資徵信名冊，梅力化學工業有限公司做了什麼？" # same as above
#input_text = "重要會計項目中，現金及約當現金的銀行存款，111年與112年分別是多少？" # loose structured table, success
#input_text = "幫我統整一下不動產、廠房及設備中，成本，折舊以及淨帳面金額" # loose structured table, poor recognize result, but success seemingly
#input_text = "請給我應收帳款(淨額)之帳齡分析的內容"
#input_text = "請給我112年，藥品進貨交易對象的名稱，金額及比率"
#input_text = "113年工作計畫有什麼社區服務相關的內容嗎？"
#input_text = "幫我總結一下113年度經費預算與上年度的比較差異"

# RAG retrieved documents
reranked_list = reranker(input_text, hybrid_retriever(vector_db, input_text, 20), threshold=0.45)
rag_docs = format_rag_output(reranked_list)

# Prompt template
prompt = f"""
# 任務
{instruction}

# 參考文件
{rag_docs}

# 用戶問題
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# llm calling
if len(rag_docs) != 0:
    response = get_completion(prompt, llm)
else:
    response = "YAh, you retrieved NOTHING!"
print(response)

==== Prompt ====

# 任務

你是台灣國泰金控的聊天機器人秘書，專門為用戶解答公司內部的差旅費用、報銷標準等相關問題。
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
3. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。


# 參考文件
Document 1:
第八條(修(修訂及施行)
附件：國內出差雜、宿、交通費標準
人員/項/項目, 1 = 全日雜費. 人員/項/項目, 2 = 宿費. 人員/項/項目, 3 = 交通費. 總經理以上, 1 = 檢據實支. 總經理以上, 2 = 檢據實支. 總經理以上, 3 = 檢據實支. 高階主管, 1 = 檢據實支. 高階主管, 2 = 檢據實支. 高階主管, 3 = 檢據實支. 部室主管及相當職級, 1 = . 部室主管及相當職級, 2 = 4 , 000(5,500)元. 部室主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 科主管及相當職級, 1 = . 科主管及相當職級, 2 = 4 , 000(5,500)元. 科主管及相當職級, 3 = 1. 以飛機經濟艙、高鐵 標準車廂或自強號之 價格為上限。 2. 如自行開車者，依自. 其他人員, 1 = . 其他人員, 2 = 4 , 000(5,500)元. 其他人員, 3 = 車 費 車 費 用 單 程 上 限上 限

Document 2:
第四條(雜宿費標準)
員工出差之雜費、宿費悉依附件標準支給之。
第五條(其他限制) 員工出差除單程旅程達 100 公里以上得申請宿費外，應以當天返回任所為原則；交通費按本
要點規定支給。

Document 3:
第八條(修(修訂及施行)
備註：
-、本公司員工於國內出差時應檢具實支，金額上限如前表；無法檢具實支者，則以 金額上限支給。
二、本要點所稱「高階主管」係指部室主管(不含)以上，總經理(不含)以下之主管。
三、員工出差如搭乘飛機或高鐵，可縮短時間、減少雜費與宿費而節省出差旅

In [ ]:
text = """

==== Prompt ====

# 任務

你是台灣國泰金控的聊天機器人秘書，專門為用戶解答公司內部的差旅費用、報銷標準等相關問題。
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
3. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。


# 參考文件
Document 1:
民國 112 年度
單位：新台幣
1, 交易人名稱 = 裕利股份有限公司. 1, 金額 = $1,230,544,978. 1, 應付票據、  帳款餘額 = $538,625,956. 1, 占該項支  出之比率 = 45%. 2, 交易人名稱 = 台灣大昌華嘉股份有限公司. 2, 金額 = 425,136,734. 2, 應付票據、  帳款餘額 = 177,372,803. 2, 占該項支  出之比率 = 15%. 3, 交易人名稱 = 久裕企業股份有限公司. 3, 金額 = 163,077,219. 3, 應付票據、  帳款餘額 = 67,343,897. 3, 占該項支  出之比率 = 6%. 4, 交易人名稱 = 華安藥品股份有限公司. 4, 金額 = 66,678,740. 4, 應付票據、  帳款餘額 = 29,857,859. 4, 占該項支  出之比率 = 2%. 5, 交易人名稱 = 台灣東洋藥品工業股份有限公司. 5, 金額 = 45,251,402. 5, 應付票據、  帳款餘額 = 16,630,969. 5, 占該項支  出之比率 = 2%. 6, 交易人名稱 = 台田藥品股份有限公司. 6, 金額 = 44,433,615. 6, 應付票據、  帳款餘額 = 17,112,370. 6, 占該項支  出之比率 = 2%. 7, 交易人名稱 = 台灣中外製藥股份有限公司. 7, 金額 = 41,218,669. 7, 應付票據、  帳款餘額 = 15,854,114. 7, 占該項支  出之比率 = 1%. 8, 交易人名稱 = 瑞安國際股份有限公司. 8, 金額 = 31,866,988. 8, 應付票據、  帳款餘額 = 9,608,150. 8, 占該項支  出之比率 = 1%. 9, 交易人名稱 = 大隆興藥品股份有限公司. 9, 金額 = 29,255,417. 9, 應付票據、  帳款餘額 = 11,576,886. 9, 占該項支  出之比率 = 1%. 10, 交易人名稱 = 元英企業股份有限公司. 10, 金額 = 28,909,315. 10, 應付票據、  帳款餘額 = 9,644,946. 10, 占該項支  出之比率 = 1%

Document 2:
民國 111 年度
單位：新台幣
醫務收入, 交易對象 = 國泰人壽. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = $9,318,908. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = $-. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 國泰金控. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 781,800. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 霖園公寓. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 208,000. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 國泰世華. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 1,400,700. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務成本-  人事費用, 交易對象 = 國泰人壽. 醫務成本-  人事費用, 關係 = 其他關係人. 醫務成本-  人事費用, 交易金額 = 55,580,357. 醫務成本-  人事費用, 授信期間 = 60 天. 醫務成本-  人事費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  人事費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  人事費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 國泰產險. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 4,229,871. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  事務費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  事務費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 國泰人壽. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 3,144,672. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  事務費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  事務費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 霖園公寓. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 5,427,619. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = 5,699,000. 醫務成本-  事務費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = 1.13%. 醫務成本-  事務費用, 備註 = 其他應付款. 醫務成本-  租金支出, 交易對象 = 國泰人壽. 醫務成本-  租金支出, 關係 = 其他關係人. 醫務成本-  租金支出, 交易金額 = 189,425,794. 醫務成本-  租金支出, 授信期間 = 不適用. 醫務成本-  租金支出, 應收(付)票據、帳款.餘額 = -. 醫務成本-  租金支出, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  租金支出, 備註 = 其他應付款. 其他營運費用, 交易對象 = 神坊資訊. 其他營運費用, 關係 = 其他關係人. 其他營運費用, 交易金額 = 6,942,134. 其他營運費用, 授信期間 = 60 天. 其他營運費用, 應收(付)票據、帳款.餘額 = 380,462. 其他營運費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = 0.08%. 其他營運費用, 備註 = 其他應付款

Document 3:
民國 111 年度
單位：新台幣
醫務收入, 交易對象 = 國泰人壽. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = $9,318,908. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = $-. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 國泰金控. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 781,800. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 霖園公寓. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 208,000. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 國泰世華. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 1,400,700. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務成本-  人事費用, 交易對象 = 國泰人壽. 醫務成本-  人事費用, 關係 = 其他關係人. 醫務成本-  人事費用, 交易金額 = 55,580,357. 醫務成本-  人事費用, 授信期間 = 60 天. 醫務成本-  人事費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  人事費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  人事費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 國泰產險. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 4,229,871. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  事務費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  事務費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 國泰人壽. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 3,144,672. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  事務費用, 應收(付)票據、

# 用戶問題
請給我112年，藥品進貨交易對象的資訊

================
<think>
好的，我现在需要回答用户的问题：“請給我112年，藥品進貨交易對象的資訊”。根据任务要求，我要参考提供的三个文件来寻找相关信息。

首先，我查看了Document 1。这个文档标题是“民國 112 年度”，里面列出了多个交易人的名称、金額、應付票據、帳款餘額以及占該項支出的比率。看起来这些交易人主要是药品公司，比如裕利股份有限公司、台湾大昌华嘉股份有限公司等。因此，Document 1提供了关于112年藥品進貨交易对象的信息。

接下来，我检查了Document 2和Document 3。这两个文档都是“民國 111 年度”的，内容涉及医务收入和成本，包括交易对象如国泰人壽、国泰金控等，但这些与药品进货无关，而是关于保险或其他服务的交易。因此，这两个文件对用户的问题没有帮助，可以忽略。

综上所述，只有Document 1提供了112年藥品進貨交易对象的具体信息。
</think>

Document 1 提供了民國 112 年度藥品進貨交易對象的資訊：

1. 裕利股份有限公司  
2. 台灣大昌華嘉股份有限公司  
3. 久裕企業股份有限公司  
4. 華安藥品股份有限公司  
5. 台湾東洋藥品工業股份有限公司  
6. 台田藥品股份有限公司  
7. 台灣中外製药股份有限公司  
8. 瑞安國際股份有限公司  
9. 大隆興藥品股份有限公司  
10. 元英企業股份有限公司
"""
len(tokenizer.tokenize(text))